In [1]:
import os
import csv
import torch
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import shutil
from jiwer import wer, cer
from datasets import Dataset
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from transformers import TrainingArguments, Trainer
import transformers

from P2_Transformers_Speech_Classification.create_datasets import create_silence_detection_list, create_dataset, \
    train_test_split_silence_detection, create_dataset_silence_classification
from project_2 import *
from transformers import TrainerCallback, TrainerControl

In [2]:
os.getcwd()

'C:\\Users\\wojew\\Documents\\DS\\sem1\\Deep Learning\\CINIC10_Proj1\\P2_Transformers_Speech_Classification'

# Load model

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
device = 'cuda' if torch.cuda.is_available() else 'cpu'
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
model.to(device)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

### Predict single audio

In [3]:
predict_single_audio(model, processor, 'data/train/audio/happy/0ab3b47d_nohash_0.wav')

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

# Dataset
### Create & save dataset

In [3]:
audio_dir = "data/train/audio/" 

#### Datasets without silence

In [ ]:
audio_dir = "data/train/audio/" 
dataset_train = create_dataset(audio_dir, processor, 'training_list')
dataset_train.save_to_disk('./dataset_train')

In [ ]:
dataset_valid = create_dataset(audio_dir, processor, 'validation_list')
dataset_valid.save_to_disk('./dataset_valid')

In [ ]:
dataset_test = create_dataset(audio_dir, processor, 'testing_list')
dataset_test.save_to_disk('./dataset_test')

#### Silence detection datasets

In [ ]:
create_silence_detection_list()
train_test_split_silence_detection()

In [4]:
dataset_train_silence_detection2 = create_dataset_silence_classification(audio_dir, processor, 'train_silence_detection_list')
dataset_train_silence_detection2.save_to_disk('./dataset_train_silence_detection2')

C:\Users\wojew\Documents\DS\venv\Lib\site-packages\transformers\models\wav2vec2\processing_wav2vec2.py:156: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(


Saving the dataset (0/1 shards):   0%|          | 0/3701 [00:00<?, ? examples/s]

In [5]:
dataset_test_silence_detection2 = create_dataset_silence_classification(audio_dir, processor, 'test_silence_detection_list')
dataset_test_silence_detection2.save_to_disk('./dataset_test_silence_detection2')

Saving the dataset (0/1 shards):   0%|          | 0/925 [00:00<?, ? examples/s]

### Load dataset

In [13]:
dataset_train = Dataset.load_from_disk("./dataset_train")
dataset_valid = Dataset.load_from_disk("./dataset_valid")
dataset_test = Dataset.load_from_disk("./dataset_test")

FileNotFoundError: No such files: 'C:/Users/wojew/Documents/DS/sem1/Deep Learning/CINIC10_Proj1/P2_Transformers_Speech_Classification/dataset_train/dataset_info.json', nor 'C:/Users/wojew/Documents/DS/sem1/Deep Learning/CINIC10_Proj1/P2_Transformers_Speech_Classification/dataset_train/state.json' found. Expected to load a `Dataset` object but provided path is not a `Dataset`.

In [10]:
dataset_test_silence_detection = Dataset.load_from_disk("./dataset_test_silence_detection")
dataset_train_silence_detection = Dataset.load_from_disk("./dataset_train_silence_detection")

In [7]:
dataset_train_silence_detection[2]

{'input_values': [-0.0007706154719926417,
  -3.739427847904153e-05,
  -0.0015038367127999663,
  0.0006958268932066858,
  0.0017956584924831986,
  0.0003292163019068539,
  0.0006958268932066858,
  0.0003292163019068539,
  0.0006958268932066858,
  0.0003292163019068539,
  -3.739427847904153e-05,
  -0.0011372260050848126,
  -0.0007706154719926417,
  0.0010624374262988567,
  -3.739427847904153e-05,
  -3.739427847904153e-05,
  0.0003292163019068539,
  -0.00040400485158897936,
  -0.00040400485158897936,
  -0.00040400485158897936,
  -0.0011372260050848126,
  0.0006958268932066858,
  0.0017956584924831986,
  -3.739427847904153e-05,
  0.0006958268932066858,
  0.0003292163019068539,
  -0.0011372260050848126,
  -0.0015038367127999663,
  -0.0007706154719926417,
  -0.00040400485158897936,
  0.0003292163019068539,
  0.0006958268932066858,
  -0.002603668486699462,
  -0.0033368896692991257,
  -0.0007706154719926417,
  0.0003292163019068539,
  -3.739427847904153e-05,
  -0.0007706154719926417,
  -3.7394

# Original model inference

In [12]:
predict_on_dataset(model, processor, dataset_valid)

NameError: name 'dataset_valid' is not defined

# Fine-tune model

In [6]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor, Trainer, TrainingArguments

model_name = "facebook/wav2vec2-base-960h"
processor = Wav2Vec2Processor.from_pretrained(model_name)
model = Wav2Vec2ForCTC.from_pretrained(model_name)
model.freeze_feature_extractor()
model.to(device)

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You sho

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureEncoder(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (activation): GELUActivation()
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
        (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
          (activation): GELUActivation()
        )
      )
    )
    (feature_projection): Wav2Vec2FeatureProjection(
      (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (projection): Linear(in_features=512, out_features=768, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder)

In [5]:
shutil.rmtree('./logs', ignore_errors=True)

In [5]:
training_args = TrainingArguments(
    output_dir="./output_dir",
    per_device_train_batch_size=16,
    num_train_epochs=20,
    logging_dir="./logs",
    save_strategy='epoch', 
    evaluation_strategy='epoch'
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    labels = [list(item[0]) for item in labels]
    if all(isinstance(i, list) for i in preds):
        preds = [item for sublist in preds for item in sublist]
    pred_transcriptions = processor.batch_decode(preds)
    true_transcriptions = processor.batch_decode(labels)
    wer_score = wer(true_transcriptions, pred_transcriptions)
    cer_score = cer(true_transcriptions, pred_transcriptions)

    correct_predictions = sum([true == pred for true, pred in zip(true_transcriptions, pred_transcriptions)])
    total_predictions = len(true_transcriptions)
    accuracy = correct_predictions / total_predictions

    available_labels = get_available_labels('data/train/audio/')
    cer_score_closest, accuracy_closest = compute_metrics_with_closest_label(available_labels, pred_transcriptions, true_transcriptions)

    return {"wer": wer_score, "accuracy": accuracy, "cer": cer_score, "accuracy_closest": accuracy_closest, "cer_closest": cer_score_closest}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_valid,
    tokenizer=processor,
    compute_metrics=compute_metrics,
    callbacks=[SaveMetricsCallback(dataset_valid)]
)

trainer.train()

 23%|██▎       | 500/2140 [01:52<05:51,  4.66it/s]

{'loss': -89.1036, 'grad_norm': 817.4182739257812, 'learning_rate': 3.831775700934579e-05, 'epoch': 1.17}


                                                  
 23%|██▎       | 500/2140 [03:23<05:51,  4.66it/s]

{'eval_loss': -68.21033477783203, 'eval_wer': 0.19123271550456017, 'eval_accuracy': 0.8087672844954399, 'eval_cer': 0.08608291901220363, 'eval_accuracy_closest': 0.8218593704030597, 'eval_cer_closest': 0.09923752083946341, 'eval_runtime': 90.5081, 'eval_samples_per_second': 75.109, 'eval_steps_per_second': 9.391, 'epoch': 1.17}


 47%|████▋     | 1000/2140 [05:15<04:21,  4.36it/s]  

...

                                                   
 93%|█████████▎| 2000/2140 [13:32<00:32,  4.34it/s]

{'eval_loss': -74.54210662841797, 'eval_wer': 0.16946160635481025, 'eval_accuracy': 0.8305383936451898, 'eval_cer': 0.07018120557176316, 'eval_accuracy_closest': 0.8402471315092674, 'eval_cer_closest': 0.07810385407472745, 'eval_runtime': 90.122, 'eval_samples_per_second': 75.431, 'eval_steps_per_second': 9.432, 'epoch': 4.67}


100%|██████████| 2140/2140 [14:04<00:00,  2.53it/s]  

{'train_runtime': 844.4191, 'train_samples_per_second': 40.472, 'train_steps_per_second': 2.534, 'train_loss': -117.87539610105141, 'epoch': 5.0}


TrainOutput(global_step=2140, training_loss=-117.87539610105141, metrics={'train_runtime': 844.4191, 'train_samples_per_second': 40.472, 'train_steps_per_second': 2.534, 'total_flos': 3.09695446656e+17, 'train_loss': -117.87539610105141, 'epoch': 5.0})

In [16]:
metrics = trainer.evaluate(dataset)
print(metrics)

100%|█████████▉| 8090/8091 [19:57<00:00, 12.95it/s]

['BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BIRD', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', '', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BERD', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BERD', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED', 'BED'

100%|██████████| 8091/8091 [20:02<00:00,  6.73it/s]

{'eval_loss': -78.64788818359375, 'eval_wer': 0.1588356174966394, 'eval_accuracy': 0.8411643825033606, 'eval_runtime': 1203.0807, 'eval_samples_per_second': 53.796, 'eval_steps_per_second': 6.725, 'epoch': 1.0}


# Adjust model - different learning rates 

In [6]:
def train_adjusted_model(lr):
    training_args = TrainingArguments(
        output_dir="./output_dir",
        per_device_train_batch_size=16,
        num_train_epochs=20,
        logging_dir="./logs",
        save_strategy='epoch', 
        evaluation_strategy='epoch',
        learning_rate=lr
    )

    def compute_metrics(pred):
        labels = pred.label_ids
        preds = pred.predictions.argmax(-1)

        labels = [list(item[0]) for item in labels]
        if all(isinstance(i, list) for i in preds):
            preds = [item for sublist in preds for item in sublist]
        pred_transcriptions = processor.batch_decode(preds)
        true_transcriptions = processor.batch_decode(labels)
        wer_score = wer(true_transcriptions, pred_transcriptions)
        cer_score = cer(true_transcriptions, pred_transcriptions)

        correct_predictions = sum([true == pred for true, pred in zip(true_transcriptions, pred_transcriptions)])
        total_predictions = len(true_transcriptions)
        accuracy = correct_predictions / total_predictions

        available_labels = get_available_labels('data/train/audio/')
        cer_score_closest, accuracy_closest = compute_metrics_with_closest_label(available_labels, pred_transcriptions, true_transcriptions)

        return {"wer": wer_score, "accuracy": accuracy, "cer": cer_score, "accuracy_closest": accuracy_closest, "cer_closest": cer_score_closest}


    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_valid,
        tokenizer=processor,
        compute_metrics=compute_metrics,
        callbacks=[SaveMetricsCallback(dataset_valid, lr)],
    )

    trainer.train()
    trainer.save_model(f"model_lr_{lr}")

In [7]:
lrs = [0.0001, 0.000005, 0.00001, 0.000005]
for lr in lrs:
    train_adjusted_model(lr)

In [9]:
dataset_test_silence_detection[5]

{'input_values': [-0.03395629674196243,
  -0.051995206624269485,
  -0.057767655700445175,
  -0.08085745573043823,
  -0.08302212506532669,
  -0.07869278639554977,
  -0.057767655700445175,
  -0.04910898208618164,
  -0.05343831703066826,
  -0.06354010850191116,
  -0.07652811706066132,
  -0.06426165997982025,
  -0.06354010850191116,
  -0.07364188879728317,
  -0.08518679440021515,
  -0.08518679440021515,
  -0.06281854957342148,
  -0.04405808448791504,
  -0.02313295565545559,
  -0.015195838175714016,
  -0.026019180193543434,
  -0.031070074066519737,
  -0.03972874954342842,
  -0.04189341515302658,
  -0.039007190614938736,
  -0.0512736514210701,
  -0.03756408020853996,
  -0.03612096607685089,
  -0.0303485170006752,
  -0.018082063645124435,
  -0.035399410873651505,
  -0.015917394310235977,
  -0.028183847665786743,
  -0.027462292462587357,
  -0.005094051361083984,
  -0.02674073539674282,
  -0.019525175914168358,
  -0.027462292462587357,
  -0.039007190614938736,
  -0.026019180193543434,
  -0.0166

# Silence detection

In [8]:
training_args = TrainingArguments(
    output_dir="./output_dir",
    per_device_train_batch_size=16,
    num_train_epochs=1,
    logging_dir="./logs",
    save_strategy='epoch', 
    evaluation_strategy='epoch',
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    labels = [list(item[0]) for item in labels]
    if all(isinstance(i, list) for i in preds):
        preds = [item for sublist in preds for item in sublist]
    pred_transcriptions = processor.batch_decode(preds)
    true_transcriptions = processor.batch_decode(labels)
    # true_transcriptions = ['' if label == 'SILENCE' else label for label in true_transcriptions]
    print(pred_transcriptions)
    print(true_transcriptions)
    wer_score = wer(true_transcriptions, pred_transcriptions)
    cer_score = cer(true_transcriptions, pred_transcriptions)

    correct_predictions = sum([true == pred for true, pred in zip(true_transcriptions, pred_transcriptions)])
    total_predictions = len(true_transcriptions)
    accuracy = correct_predictions / total_predictions

    available_labels = get_available_labels('data/train/audio/')
    cer_score_closest, accuracy_closest = compute_metrics_with_closest_label(available_labels, pred_transcriptions, true_transcriptions)

    return {"wer": wer_score, "accuracy": accuracy, "cer": cer_score, "accuracy_closest": accuracy_closest, "cer_closest": cer_score_closest}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_silence_detection,
    eval_dataset=dataset_test_silence_detection,
    tokenizer=processor,
    compute_metrics=compute_metrics,
    callbacks=[SaveMetricsCallback(dataset_test_silence_detection, 'silence_detection')],
)

trainer.train()
trainer.save_model(f"model_silence_detection")

C:\Users\wojew\Documents\DS\venv\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


['SOUND', 'SOUND', '', 'SOUND', 'SOUND', 'SOUND', '', '', '', 'SOUND', '', 'SOUND', 'SOUND', 'SOUND', 'SOUND', '', 'SOUND', '', 'SOUND', 'SOUND', '', '', '', 'SOUND', '', '', '', 'SOUND', '', 'SOUND', '', '', '', '', 'SOUND', 'SOUND', '', '', 'SOUND', '', '', 'SOUND', 'SOUND', '', '', 'SOUND', '', 'SOUND', '', '', 'SOUND', 'SOUND', 'SOUND', 'SOUND', '', '', 'SOUND', 'SOUND', 'SOUND', '', 'SOUND', 'SOUND', '', 'SOUND', '', '', 'SOUND', 'SOUND', 'SOUND', '', 'SOUND', 'SOUND', '', '', '', '', '', 'SOUND', 'SOUND', 'SOUND', 'SOUND', 'SOUNDS', 'SOUND', 'SOUND', 'SOUND', 'SOUND', '', '', '', 'SOUND', 'SOUND', '', 'SOUND', '', 'SOUND', '', 'SOUND', 'SOUND', '', '', '', 'SOUND', 'SOUND', '', 'SOUND', 'SOUND', 'SOUND', 'SOUND', 'SOUND', 'SOUND', 'SOUND', '', '', 'SOUND', '', 'SOUND', '', 'SOUND', '', 'SOUND', '', 'SOUND', 'SOUND', '', 'SOUND', 'SOUND', '', 'SOUND', 'SOUND', '', 'SOUND', 'SOUND', '', 'SOUND', '', 'SOUND', 'SOUND', '', 'SOUND', '', '', 'SOUND', 'SOUND', 'SOUND', 'SOUND', '', '', 

In [ ]:
training_args = TrainingArguments(
    output_dir="./output_dir",
    per_device_train_batch_size=16,
    num_train_epochs=1,
    logging_dir="./logs",
    save_strategy='epoch', 
    evaluation_strategy='epoch',
)

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    labels = [list(item[0]) for item in labels]
    if all(isinstance(i, list) for i in preds):
        preds = [item for sublist in preds for item in sublist]
    pred_transcriptions = processor.batch_decode(preds)
    true_transcriptions = processor.batch_decode(labels)
    # true_transcriptions = ['' if label == 'SILENCE' else label for label in true_transcriptions]
    print(pred_transcriptions)
    print(true_transcriptions)
    wer_score = wer(true_transcriptions, pred_transcriptions)
    cer_score = cer(true_transcriptions, pred_transcriptions)

    correct_predictions = sum([true == pred for true, pred in zip(true_transcriptions, pred_transcriptions)])
    total_predictions = len(true_transcriptions)
    accuracy = correct_predictions / total_predictions

    available_labels = get_available_labels('data/train/audio/')
    cer_score_closest, accuracy_closest = compute_metrics_with_closest_label(available_labels, pred_transcriptions, true_transcriptions)

    return {"wer": wer_score, "accuracy": accuracy, "cer": cer_score, "accuracy_closest": accuracy_closest, "cer_closest": cer_score_closest}


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train_silence_detection2,
    eval_dataset=dataset_test_silence_detection2,
    tokenizer=processor,
    compute_metrics=compute_metrics,
    callbacks=[SaveMetricsCallback(dataset_test_silence_detection2, 'silence_detection_2')],
)

trainer.train()
trainer.save_model(f"model_silence_detection2")

C:\Users\wojew\Documents\DS\venv\Lib\site-packages\accelerate\accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:
# zapisywanie scieżek do plików nie-silence

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    labels = [list(item[0]) for item in labels]
    if all(isinstance(i, list) for i in preds):
        preds = [item for sublist in preds for item in sublist]
    pred_transcriptions = processor.batch_decode(preds)
    true_transcriptions = processor.batch_decode(labels)

    # List to store file paths of non-'silence' predictions
    non_silence_files = []

    for i, pred_transcription in enumerate(pred_transcriptions):
        if pred_transcription != 'silence':
            non_silence_files.append(training_list[i])  # assuming training_list is accessible and contains file paths

    # Write non-'silence' file paths to a text file
    with open('non_silence_files.txt', 'w') as f:
        for file_path in non_silence_files:
            f.write(f'{file_path}\n')

    # ... rest of the function

In [ ]:
import os
import librosa
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

# Load the trained model and the processor
model = Wav2Vec2ForCTC.from_pretrained("path_to_trained_model")
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

# Directory containing the new, unseen audio files
audio_dir = "path_to_new_audio_files"

# List to store file paths of non-'silence' predictions
non_silence_files = []

# Process each audio file in the directory
for audio_file in os.listdir(audio_dir):
    # Load the audio file
    audio, rate = librosa.load(os.path.join(audio_dir, audio_file), sr=16000)

    # Process the audio file with the processor
    inputs = processor(audio, return_tensors="pt", padding=True, sampling_rate=16000)

    # Make a prediction using the model
    with torch.no_grad():
        logits = model(inputs.input_values.to("cuda")).logits

    # Decode the prediction to get the transcription
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = processor.decode(predicted_ids[0])

    # If the transcription is not 'silence', append the file path to the list
    if transcription != 'silence':
        non_silence_files.append(os.path.join(audio_dir, audio_file))

# Write non-'silence' file paths to a text file
with open('non_silence_files.txt', 'w') as f:
    for file_path in non_silence_files:
        f.write(f'{file_path}\n')

# Classification with silence as class